**做了四個方法**
**效果都很差**

1.SVM

2.SVM + SMOTE

3.SVM + PCA_80_35 

4.SVM + PCA_90_44

5.SVM + PCA_80_35 + SMOTE

6.SVM + PCA_90_44 + SMOTE

# import

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
import warnings

# read file

In [2]:
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')

# 1. SVM

沒有使用平衡處理的情況下，所有目標都被當成0

In [24]:
# ===== 1. 特徵與目標 =====
X = filled_data_label_numeric_df.drop(columns=['abnormal_target'])
y = filled_data_label_numeric_df['abnormal_target']

# ===== 2. 資料標準化 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 建立 SVM 模型 =====
svm_model = SVC(kernel='rbf')  
svm_model.fit(X_train, y_train)

# ===== 5. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 6. 評估模型表現 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))
print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

📊 混淆矩陣：
[[796   0]
 [  8   0]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      1.00      0.99       796
Abnormal (1)       0.00      0.00      0.00         8

    accuracy                           0.99       804
   macro avg       0.50      0.50      0.50       804
weighted avg       0.98      0.99      0.99       804



c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

# 2. SVM + SMOTE

成功抓出來的1從0變成2，但還是有6個1沒有被抓出來

In [32]:
# ===== 1. 特徵與目標 =====
X = filled_data_label_numeric_df.drop(columns=['abnormal_target'])
y = filled_data_label_numeric_df['abnormal_target']

# ===== 2. 資料標準化 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 套用 SMOTE 做過採樣（只對訓練集做）=====
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# ===== 5. 建立 SVM 模型 =====
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_resampled, y_train_resampled)

# ===== 6. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 7. 評估模型表現 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))
print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

📊 混淆矩陣：
[[796   0]
 [  6   2]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      1.00      1.00       796
Abnormal (1)       1.00      0.25      0.40         8

    accuracy                           0.99       804
   macro avg       1.00      0.62      0.70       804
weighted avg       0.99      0.99      0.99       804



# 3. SVM + PCA80_35

用了PCA後，一樣都變認不出來

In [ ]:
PCA80_35_df = pd.read_csv("../data/PCA80_35.csv")
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')

In [45]:
# ===== 1. 特徵與目標（用 PCA80_35 當 X，原始表單取 y）=====
X = PCA80_35_df  # 降維後的特徵
y = filled_data_label_numeric_df['abnormal_target']  # 原始目標值

# ===== 2. 資料標準化（視情況你也可以不做，但通常會更穩定）=====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 建立 SVM 模型（不加 class_weight）=====
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

# ===== 5. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 6. 評估 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))

print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

print("🎯 F1-score（macro）:", f1_score(y_test, y_pred, average='macro'))

📊 混淆矩陣：
[[796   0]
 [  8   0]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      1.00      0.99       796
Abnormal (1)       0.00      0.00      0.00         8

    accuracy                           0.99       804
   macro avg       0.50      0.50      0.50       804
weighted avg       0.98      0.99      0.99       804

🎯 F1-score（macro）: 0.4975


# 4. SVM + PCA90_44

用了PCA後，一樣都變認不出來

In [ ]:
PCA90_44_df = pd.read_csv("../data/PCA90_44.csv")
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')

In [48]:
# ===== 1. 特徵與目標（用 PCA80_35 當 X，原始表單取 y）=====
X = PCA90_44_df  # 降維後的特徵
y = filled_data_label_numeric_df['abnormal_target']  # 原始目標值

# ===== 2. 資料標準化（視情況你也可以不做，但通常會更穩定）=====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 建立 SVM 模型（不加 class_weight）=====
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

# ===== 5. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 6. 評估 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))

print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

print("🎯 F1-score（macro）:", f1_score(y_test, y_pred, average='macro'))

📊 混淆矩陣：
[[796   0]
 [  8   0]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      1.00      0.99       796
Abnormal (1)       0.00      0.00      0.00         8

    accuracy                           0.99       804
   macro avg       0.50      0.50      0.50       804
weighted avg       0.98      0.99      0.99       804

🎯 F1-score（macro）: 0.4975


# 5. SVM + PCA80_35 + SMOTE

比不做SMOTE好一丟丟

In [49]:
PCA80_35_df = pd.read_csv("../data/PCA80_35.csv")
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')

In [50]:
# ===== 1. 特徵與目標（用 PCA80_35 當 X，原始表單取 y）=====
X = PCA80_35_df
y = filled_data_label_numeric_df['abnormal_target']

# ===== 2. 資料標準化 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 套用 SMOTE 過採樣（只對訓練資料做）=====
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# ===== 5. 建立 SVM 模型 =====
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train_resampled, y_train_resampled)

# ===== 6. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 7. 評估 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))

print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

print("🎯 F1-score（macro）:", f1_score(y_test, y_pred, average='macro'))

📊 混淆矩陣：
[[792   4]
 [  7   1]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      0.99      0.99       796
Abnormal (1)       0.20      0.12      0.15         8

    accuracy                           0.99       804
   macro avg       0.60      0.56      0.57       804
weighted avg       0.98      0.99      0.98       804

🎯 F1-score（macro）: 0.573474801061008


# 6. SVM + PCA90_44 + SMOTE

比不做SMOTE好一丟丟

In [51]:
PCA90_44_df = pd.read_csv("../data/PCA90_44.csv")
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')

In [53]:
# ===== 1. 特徵與目標（用 PCA80_35 當 X，原始表單取 y）=====
X = PCA90_44_df
y = filled_data_label_numeric_df['abnormal_target']

# ===== 2. 資料標準化 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== 3. 切分訓練與測試資料 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ===== 4. 套用 SMOTE 過採樣（只對訓練資料做）=====
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# ===== 5. 建立 SVM 模型 =====
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train_resampled, y_train_resampled)

# ===== 6. 預測 =====
y_pred = svm_model.predict(X_test)

# ===== 7. 評估 =====
print("📊 混淆矩陣：")
print(confusion_matrix(y_test, y_pred))

print("\n📄 分類報告：")
print(classification_report(y_test, y_pred, target_names=["Normal (0)", "Abnormal (1)"]))

print("🎯 F1-score（macro）:", f1_score(y_test, y_pred, average='macro'))

📊 混淆矩陣：
[[795   1]
 [  7   1]]

📄 分類報告：
              precision    recall  f1-score   support

  Normal (0)       0.99      1.00      0.99       796
Abnormal (1)       0.50      0.12      0.20         8

    accuracy                           0.99       804
   macro avg       0.75      0.56      0.60       804
weighted avg       0.99      0.99      0.99       804

🎯 F1-score（macro）: 0.5974968710888611
